In [38]:
import pandas as pd
import numpy as np

In [39]:
def getIndMatrix(barIx, t1):
    # Indicator matrix from bar index and t1
    indM = pd.DataFrame(0, index=barIx, columns=range(t1.shape[0]))
    for i, (t0, t1_val) in enumerate(t1.items()):
        indM.loc[t0:t1_val, i] = 1.
    return indM

In [46]:
def getAvgUniqueness(indM):
    # Average Uniqueness from indicator matrix
    c = indM.sum(axis=1) # concurrent events
    u = indM.div(c, axis=0) # uniqueness matrix
    avgU = u.mean().values # average uniqueness
    return avgU

In [43]:
# Set up barIx (time index)
barIx = pd.Index(range(7), name='t')

# Set up t1 as a Series where:
# - index is the start time (t0)
# - values are the end times (t1)
t1 = pd.Series([3, 4, 6], index=[0, 3, 4])


In [44]:
indM = getIndMatrix(barIx, t1)
print(indM)

   0  1  2
t         
0  1  0  0
1  1  0  0
2  1  0  0
3  1  1  0
4  0  1  1
5  0  0  1
6  0  0  1


In [48]:
avgU = getAvgUniqueness(indM)
print("Average Uniqueness:", avgU)

Average Uniqueness: [0.5        0.14285714 0.35714286]
